In [2]:
!git clone https://github.com/tungduong0708/G3-Original.git

Cloning into 'G3-Original'...
remote: Enumerating objects: 348, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 348 (delta 68), reused 69 (delta 28), pack-reused 238 (from 1)
Receiving objects: 100% (348/348), 37.67 MiB | 38.31 MiB/s, done.
Resolving deltas: 100% (222/222), done.


In [3]:
%cd G3-Original

/kaggle/working/G3-Original


In [52]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu124
!pip install transformers accelerate huggingface_hub pandas optuna pyproj einops geopy matplotlib kaggle  geopandas cartopy

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 2.1 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 2.6 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 50.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 73.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 4.7 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import snapshot_download, hf_hub_download, login

login(token="")

path = snapshot_download(
    repo_id="tduongvn/ACMMM25-Geolocation",
    repo_type="dataset",
    allow_patterns=["*.tar"],
    local_dir="data/mp16/",
    use_auth_token=True  # will use your local token from `huggingface-cli login`
)
print(f"Downloaded dataset to {path}")

files = [
    "metadata/MP16_Pro_filtered.csv",
    "metadata/MP16_Pro_places365.csv",
    "metadata/mp16_urls.csv"
]
for file in files:
    path = hf_hub_download(
        repo_id="Jia-py/MP16-Pro",
        filename=file,
        repo_type="dataset",
        local_dir="data/mp16/",
        use_auth_token=True  # will use your local token from `huggingface-cli login`
    )
    print(f"Downloaded {file} to {path}")


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

filtered_mp16.tar:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

Downloaded dataset to /kaggle/working/G3-Original/data/mp16


MP16_Pro_filtered.csv:   0%|          | 0.00/747M [00:00<?, ?B/s]

Downloaded metadata/MP16_Pro_filtered.csv to data/mp16/metadata/MP16_Pro_filtered.csv


MP16_Pro_places365.csv:   0%|          | 0.00/859M [00:00<?, ?B/s]

Downloaded metadata/MP16_Pro_places365.csv to data/mp16/metadata/MP16_Pro_places365.csv


mp16_urls.csv:   0%|          | 0.00/385M [00:00<?, ?B/s]

Downloaded metadata/mp16_urls.csv to data/mp16/metadata/mp16_urls.csv


In [54]:
!mv data/mp16/metadata/*.csv data/mp16

In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_KEY'] = ''

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

api.dataset_download_files('lctngdng/im2gps3k', path='data/im2gps3k', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/lctngdng/im2gps3k


In [56]:
import geopandas as gpd
from shapely.geometry import Point
import cartopy.io.shapereader as shpreader

# Load Natural Earth country polygons
shp_path = shpreader.natural_earth(
    resolution='110m',
    category='cultural',
    name='admin_0_countries'
)
world = gpd.read_file(shp_path)[["ADMIN", "geometry"]].to_crs(epsg=4326)
# Build spatial index for performance
sindex = world.sindex

# Function returns country name for a given latitude/longitude
def point_to_country(lat, lon, countries=world, index=sindex):
    pt = Point(lon, lat)
    # Find candidate polygons via spatial index
    candidates = list(index.intersection(pt.bounds))
    # Check strict land contains
    for idx in candidates:
        if countries.geometry.iloc[idx].contains(pt):
            return countries.ADMIN.iloc[idx]
    # Fallback to intersects for border/water cases
    for idx in candidates:
        if countries.geometry.iloc[idx].intersects(pt):
            return countries.ADMIN.iloc[idx]
    # No match found
    return None

In [57]:
import pandas as pd
df = pd.read_csv("data/im2gps3k/im2gps3k_places365.csv")
df.head()

,IMG_ID,AUTHOR,LAT,LON,S3_Label,S16_Label,S365_Label,Prob_indoor,Prob_natural,Prob_urban
0,1000269685_e60e9cdfb4_1125_78841376@N00.jpg,78841376@N00,32.325436,-64.764404,2,12,353,0.274242,0.045113,0.680645
1,1000304467_1a75a200b1_1296_78841376@N00.jpg,78841376@N00,32.325436,-64.764404,0,4,325,0.414407,0.220912,0.364681
2,1001048550_8e4b47d165_1051_78841376@N00.jpg,78841376@N00,32.325436,-64.764404,1,8,36,0.007326,0.969903,0.022771
3,1005977048_5ccf8b05d3_1201_91728102@N00.jpg,91728102@N00,29.976052,122.390356,2,6,273,0.003868,0.476665,0.519468
4,1008804117_ce4e6fef8a_1349_97522422@N00.jpg,97522422@N00,46.478536,30.758714,0,0,198,0.973878,0.002453,0.023670


In [58]:
def is_target_country(row):
    lat, lon = row["LAT"], row["LON"]
    return point_to_country(lat, lon) in ["Ukraine", "Israel", "Russia", "Palestine"]

# apply row‐wise and get a boolean mask
mask = df.apply(is_target_country, axis=1)

# filter the dataframe
df_filtered = df[mask]
df_filtered.head()
df_filtered.to_csv("data/im2gps3k/im2gps3k_places365.csv")

In [59]:
from pathlib import Path
import shutil
Path("data/im2gps3k/filtered_im2gps3k").mkdir(parents=True, exist_ok=True)
for image in df_filtered["IMG_ID"]:
    shutil.copyfile(
        f"data/im2gps3k/im2gps3ktest/{image}",
        f"data/im2gps3k/filtered_im2gps3k/{image}"
    )

In [ ]:
# Standard library
import os
import time
import warnings
import yaml

# Third-party libraries
import torch
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.optim.adamw import AdamW
from accelerate import Accelerator, DistributedDataParallelKwargs

# Local application imports
from utils.utils import MP16Dataset
from utils.G3 import G3
from zeroshot_prediction import ZeroShotPredictor


warnings.filterwarnings('ignore')

def train_1epoch(dataloader, eval_dataloader, earlystopper, model, vision_processor, text_processor, optimizer, scheduler, device, accelerator=None):
    model.train()
    t = tqdm(dataloader, disable=not accelerator.is_local_main_process)
    for i, (images, texts, longitude, latitude) in enumerate(t):
        texts = text_processor(text=texts, padding='max_length', truncation=True, return_tensors='pt', max_length=77)
        images = images.to(device)
        texts = texts.to(device)
        longitude = longitude.to(device).float()
        latitude = latitude.to(device).float()
        optimizer.zero_grad()

        output = model(images, texts, longitude, latitude, return_loss=True)
        loss = output['loss']

        if accelerator is not None:
            accelerator.backward(loss)
        else:
            loss.backward()
        optimizer.step()
        if i % 1 == 0:
            t.set_description('step {}, loss {}, lr {}'.format(i, loss.item(), scheduler.get_last_lr()[0]))
    scheduler.step()


2025-05-27 23:06:20.480092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748387180.506688     215 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748387180.514970     215 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
%cd G3-Original

/kaggle/working/G3-Original


In [ ]:
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)
accelerator = Accelerator(kwargs_handlers=[ddp_kwargs])

# fine-tune
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
hparams = yaml.safe_load(open('hparams.yaml', 'r'))
pe = "projection_ecef"
nn = "rffmlp"
model = G3(
    device=device,
    positional_encoding_type=pe,
    neural_network_type=nn,
    hparams=hparams[f'{pe}_{nn}'],
).to(device)

### ECEF Finetune

In [ ]:
from utils.rff.layers import GaussianEncoding
from pyproj import Proj, Transformer

location_encoder_dict = torch.load('checkpoints/location_encoder_weights.pth') # from geoclip

new_location_encoder_dict = {}
for k, v in location_encoder_dict.items():
    new_key = 'neural_network.0.' + k
    new_location_encoder_dict[new_key] = v

model.location_encoder.load_state_dict(new_location_encoder_dict)


print(model.location_encoder.neural_network[0].LocEnc0.capsule[0])
sigma = torch.tensor([1.0, 16.0, 256.0])
model.location_encoder.neural_network[0].LocEnc0.capsule[0] = GaussianEncoding(sigma=sigma, input_size=3, encoded_size=256)
model.location_encoder.neural_network[0].LocEnc1.capsule[0] = GaussianEncoding(sigma=sigma, input_size=3, encoded_size=256)
model.location_encoder.neural_network[0].LocEnc2.capsule[0] = GaussianEncoding(sigma=sigma, input_size=3, encoded_size=256)


# Manually replace the Projection to use ECEF
model.location_encoder.position_encoder.projection = "ecef"
model.location_encoder.position_encoder.normalizer = 6378137.0
model.location_encoder.position_encoder.embedding_dim = [3]
model.location_encoder.position_encoder.transformer = Transformer.from_proj(
    Proj('epsg:4326'), Proj('epsg:4978'), always_xy=True
)

dict_keys(['neural_network.0.LocEnc0.capsule.0.b', 'neural_network.0.LocEnc0.capsule.1.weight', 'neural_network.0.LocEnc0.capsule.1.bias', 'neural_network.0.LocEnc0.capsule.3.weight', 'neural_network.0.LocEnc0.capsule.3.bias', 'neural_network.0.LocEnc0.capsule.5.weight', 'neural_network.0.LocEnc0.capsule.5.bias', 'neural_network.0.LocEnc0.head.0.weight', 'neural_network.0.LocEnc0.head.0.bias', 'neural_network.0.LocEnc1.capsule.0.b', 'neural_network.0.LocEnc1.capsule.1.weight', 'neural_network.0.LocEnc1.capsule.1.bias', 'neural_network.0.LocEnc1.capsule.3.weight', 'neural_network.0.LocEnc1.capsule.3.bias', 'neural_network.0.LocEnc1.capsule.5.weight', 'neural_network.0.LocEnc1.capsule.5.bias', 'neural_network.0.LocEnc1.head.0.weight', 'neural_network.0.LocEnc1.head.0.bias', 'neural_network.0.LocEnc2.capsule.0.b', 'neural_network.0.LocEnc2.capsule.1.weight', 'neural_network.0.LocEnc2.capsule.1.bias', 'neural_network.0.LocEnc2.capsule.3.weight', 'neural_network.0.LocEnc2.capsule.3.bias', '

### SIREN-SH Finetune

In [ ]:
from huggingface_hub import hf_hub_download
import torch

hf_hub_download(
    repo_id="microsoft/SatCLIP-ViT16-L40",
    filename="satclip-vit16-l40.ckpt",
    repo_type="model",
    local_dir="./"
)


ckpt_path = "satclip-vit16-l40.ckpt"
ckpt = torch.load(ckpt_path, map_location="cuda")

if "state_dict" in ckpt:
    state_dict = ckpt["state_dict"]
else:
    state_dict = ckpt  

location_state_dict = {
    k.replace("model.location.nnet", "neural_network.0"): v for k, v in state_dict.items() if k.startswith("model.location.")
}

for k, v in location_state_dict.items():
    print(k, v.shape)
    
model.location_encoder.load_state_dict(location_state_dict)

### Start...

In [ ]:

dataset = MP16Dataset(vision_processor = model.vision_processor, text_processor = model.text_processor, root_path='data/mp16/',image_data_path='filtered_mp16.tar')
dataloader = DataLoader(dataset, batch_size=256, shuffle=False, num_workers=16, pin_memory=True, prefetch_factor=5)


params = []
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.size())
        params.append(param)

optimizer = AdamW([param for name, param in model.named_parameters() if param.requires_grad], lr=hparams[f'{pe}_{nn}']['lr'], weight_decay=hparams[f'{pe}_{nn}']['wd'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.87)

model, optimizer, dataloader, scheduler = accelerator.prepare(
    model, optimizer, dataloader, scheduler
)

eval_dataloader = None
earlystopper = None
for epoch in range(10):
    train_1epoch(dataloader, eval_dataloader, earlystopper, model, model.vision_processor, model.text_processor, optimizer, scheduler, device, accelerator)
    unwrapped_model = accelerator.unwrap_model(model)
    os.makedirs('checkpoints', exist_ok=True)
    torch.save(unwrapped_model, 'checkpoints/g3_{}_.pth'.format(epoch))

    with open("threshold_accuracy.txt", "a") as f:
        f.write('checkpoints/g3_{}_.pth\n'.format(epoch))


    predictor = ZeroShotPredictor(model='checkpoints/g3_{}_.pth'.format(epoch), device=device)
    df, res = predictor.evaluate_im2gps3k(
        df_path="data/im2gps3k/im2gps3k_places365.csv",
        top_k=5,
        root_path="data/im2gps3k/",
        image_data_path="filtered_im2gps3k",
        text_data_path= "im2gps3k_places365.csv"
    )

read text data success
no exist tar index success, need building...


42764it [00:01, 30022.27it/s]


tar index buidling success
data columns:  42764
location from str to float success
logit_scale1 torch.Size([])
logit_scale2 torch.Size([])
logit_scale3 torch.Size([])
location_encoder.neural_network.0.LocEnc0.capsule.1.weight torch.Size([1024, 512])
location_encoder.neural_network.0.LocEnc0.capsule.1.bias torch.Size([1024])
location_encoder.neural_network.0.LocEnc0.capsule.3.weight torch.Size([1024, 1024])
location_encoder.neural_network.0.LocEnc0.capsule.3.bias torch.Size([1024])
location_encoder.neural_network.0.LocEnc0.capsule.5.weight torch.Size([1024, 1024])
location_encoder.neural_network.0.LocEnc0.capsule.5.bias torch.Size([1024])
location_encoder.neural_network.0.LocEnc0.head.0.weight torch.Size([512, 1024])
location_encoder.neural_network.0.LocEnc0.head.0.bias torch.Size([512])
location_encoder.neural_network.0.LocEnc1.capsule.1.weight torch.Size([1024, 512])
location_encoder.neural_network.0.LocEnc1.capsule.1.bias torch.Size([1024])
location_encoder.neural_network.0.LocEnc1.c

  0%|          | 0/168 [00:00<?, ?it/s]

In [ ]:
!mv checkpoints/g3_5_.pth checkpoints/sirensh_6epoch_tmp.pth

In [ ]:
from huggingface_hub import upload_file

upload_file(
    path_or_fileobj="checkpoints/sirensh_6epoch_tmph.pth",        # Local file path
    path_in_repo="sirensh_6epoch_tmp.pth",                         # Desired path in the repo
    repo_id="tduongvn/Checkpoints-ACMMM25",               # e.g., "tungduong/my-model-repo"
    repo_type="model",                                    # or "dataset" or "space"
    commit_message="Upload sirensh_6epoch_tmp.pth"
)
